In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score, plot_roc_curve, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, \
AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier, RandomForestClassifier,\
ExtraTreesClassifier, VotingClassifier, StackingRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
%matplotlib inline

In [2]:
df = pd.read_csv('C:/Users/Nick/Documents/Flatiron/phase_3/project3/data1.csv')
df.head(2)

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,h1n1_vaccine,seasonal_vaccine
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0,0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,0,1


In [116]:
df.info()
df = df.drop(columns=['health_insurance', 'employment_occupation', 'employment_industry'])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 19642 entries, 0 to 26706
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   respondent_id                19642 non-null  int64  
 1   h1n1_concern                 19642 non-null  float64
 2   h1n1_knowledge               19642 non-null  float64
 3   behavioral_antiviral_meds    19642 non-null  float64
 4   behavioral_avoidance         19642 non-null  float64
 5   behavioral_face_mask         19642 non-null  float64
 6   behavioral_wash_hands        19642 non-null  float64
 7   behavioral_large_gatherings  19642 non-null  float64
 8   behavioral_outside_home      19642 non-null  float64
 9   behavioral_touch_face        19642 non-null  float64
 10  doctor_recc_h1n1             19642 non-null  float64
 11  doctor_recc_seasonal         19642 non-null  float64
 12  chronic_med_condition        19642 non-null  float64
 13  child_under_6_mo

KeyError: "['health_insurance' 'employment_occupation' 'employment_industry'] not found in axis"

In [81]:
df = df.dropna()

In [21]:
df_health = df.dropna(subset=['health_insurance'])
df_health.drop(['employment_industry', 'employment_occupation'], axis = 1, inplace=True)
df_health.dropna(inplace=True)
df_health.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11794 entries, 0 to 26706
Data columns (total 36 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   respondent_id                11794 non-null  int64  
 1   h1n1_concern                 11794 non-null  float64
 2   h1n1_knowledge               11794 non-null  float64
 3   behavioral_antiviral_meds    11794 non-null  float64
 4   behavioral_avoidance         11794 non-null  float64
 5   behavioral_face_mask         11794 non-null  float64
 6   behavioral_wash_hands        11794 non-null  float64
 7   behavioral_large_gatherings  11794 non-null  float64
 8   behavioral_outside_home      11794 non-null  float64
 9   behavioral_touch_face        11794 non-null  float64
 10  doctor_recc_h1n1             11794 non-null  float64
 11  doctor_recc_seasonal         11794 non-null  float64
 12  chronic_med_condition        11794 non-null  float64
 13  child_under_6_mo

C:\Users\Nick\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-21-e3df79885ad7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_health.dropna(inplace=True)


In [56]:
df_float = df.loc[:,df.dtypes == 'float64']
df_int = df.loc[:,df.dtypes == 'int64']
df_object = df.loc[:,df.dtypes == 'object']
df_float.shape, df_int.shape, df_object.shape

((26707, 22), (26707, 3), (26707, 10))

In [119]:
df_numeric = pd.concat([df_float, df_int], axis = 1)
df_numeric.columns

Index(['h1n1_concern', 'h1n1_knowledge', 'behavioral_antiviral_meds',
       'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'household_adults',
       'household_children', 'respondent_id', 'h1n1_vaccine',
       'seasonal_vaccine'],
      dtype='object')

In [170]:
top_feat_numeric = [x for x in df_numeric if x in top_feat_list]

In [158]:
df_cat_dummied = pd.get_dummies(df_object, drop_first=True).astype('int64')
df_cat_dummied.columns

Index(['age_group_35 - 44 Years', 'age_group_45 - 54 Years',
       'age_group_55 - 64 Years', 'age_group_65+ Years',
       'education_< 12 Years', 'education_College Graduate',
       'education_Some College', 'race_Hispanic', 'race_Other or Multiple',
       'race_White', 'sex_Male', 'income_poverty_> $75,000',
       'income_poverty_Below Poverty', 'marital_status_Not Married',
       'rent_or_own_Rent', 'employment_status_Not in Labor Force',
       'employment_status_Unemployed', 'hhs_geo_region_bhuqouqj',
       'hhs_geo_region_dqpwygqj', 'hhs_geo_region_fpwskwrf',
       'hhs_geo_region_kbazzjca', 'hhs_geo_region_lrircsnp',
       'hhs_geo_region_lzgpxyit', 'hhs_geo_region_mlyzmhmf',
       'hhs_geo_region_oxchjgsf', 'hhs_geo_region_qufhixun',
       'census_msa_MSA, Principle City', 'census_msa_Non-MSA'],
      dtype='object')

In [156]:
df_final = pd.concat([df_numeric, df_cat_dummied], axis=1)
df_top_feat=df_final[['h1n1_knowledge',
 'opinion_seas_vacc_effective',
 'rent_or_own_Rent',
 'opinion_h1n1_sick_from_vacc',
 'hhs_geo_region_lzgpxyit',
 'behavioral_large_gatherings',
 'health_worker',
 'age_group_65+ Years',
 'race_White',
 'child_under_6_months',
 'doctor_recc_h1n1',
 'sex_Male',
 'opinion_h1n1_risk',
 'income_poverty_> $75,000',
 'education_< 12 Years',
 'sex_Female',
 'marital_status_Married',
 'opinion_seas_risk',
 'employment_status_Employed',
 'opinion_seas_sick_from_vacc',
 'household_adults',
 'chronic_med_condition',
 'employment_status_Not in Labor Force',
 'opinion_h1n1_vacc_effective',
 'education_College Graduate',
 'doctor_recc_seasonal',
 'behavioral_touch_face',
 'census_msa_MSA, Not Principle  City',
 'household_children',
 'h1n1_concern',
 'income_poverty_<= $75,000, Above Poverty',
 'age_group_55 - 64 Years',
 'hhs_geo_region_bhuqouqj',
 'marital_status_Not Married',
 'seasonal_vaccine', 'h1n1_vaccine']]

In [165]:
top_feat_list = ['h1n1_knowledge',
 'opinion_seas_vacc_effective',
 'rent_or_own_Rent',
 'opinion_h1n1_sick_from_vacc',
 'hhs_geo_region_lzgpxyit',
 'behavioral_large_gatherings',
 'health_worker',
 'age_group_65+ Years',
 'race_White',
 'child_under_6_months',
 'doctor_recc_h1n1',
 'sex_Male',
 'opinion_h1n1_risk',
 'income_poverty_> $75,000',
 'education_< 12 Years',
 'sex_Female',
 'marital_status_Married',
 'opinion_seas_risk',
 'employment_status_Employed',
 'opinion_seas_sick_from_vacc',
 'household_adults',
 'chronic_med_condition',
 'employment_status_Not in Labor Force',
 'opinion_h1n1_vacc_effective',
 'education_College Graduate',
 'doctor_recc_seasonal',
 'behavioral_touch_face',
 'census_msa_MSA, Not Principle  City',
 'household_children',
 'h1n1_concern',
 'income_poverty_<= $75,000, Above Poverty',
 'age_group_55 - 64 Years',
 'hhs_geo_region_bhuqouqj',
 'marital_status_Not Married',
 'seasonal_vaccine', 'h1n1_vaccine']

In [159]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 63 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   h1n1_concern                              26615 non-null  float64
 1   h1n1_knowledge                            26591 non-null  float64
 2   behavioral_antiviral_meds                 26636 non-null  float64
 3   behavioral_avoidance                      26499 non-null  float64
 4   behavioral_face_mask                      26688 non-null  float64
 5   behavioral_wash_hands                     26665 non-null  float64
 6   behavioral_large_gatherings               26620 non-null  float64
 7   behavioral_outside_home                   26625 non-null  float64
 8   behavioral_touch_face                     26579 non-null  float64
 9   doctor_recc_h1n1                          24547 non-null  float64
 10  doctor_recc_seasonal              

In [160]:
df_final = df_final.dropna()
df_top_feat = df_top_feat.dropna()
df_top_feat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23227 entries, 0 to 26706
Data columns (total 36 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   h1n1_knowledge                            23227 non-null  float64
 1   opinion_seas_vacc_effective               23227 non-null  float64
 2   rent_or_own_Rent                          23227 non-null  int64  
 3   opinion_h1n1_sick_from_vacc               23227 non-null  float64
 4   hhs_geo_region_lzgpxyit                   23227 non-null  int64  
 5   behavioral_large_gatherings               23227 non-null  float64
 6   health_worker                             23227 non-null  float64
 7   age_group_65+ Years                       23227 non-null  int64  
 8   race_White                                23227 non-null  int64  
 9   child_under_6_months                      23227 non-null  float64
 10  doctor_recc_h1n1                  

In [151]:
df_top_feat

,h1n1_knowledge,opinion_seas_vacc_effective,rent_or_own_Rent,opinion_h1n1_sick_from_vacc,hhs_geo_region_lzgpxyit,behavioral_large_gatherings,health_worker,age_group_65+ Years,race_White,child_under_6_months,...,doctor_recc_seasonal,behavioral_touch_face,"census_msa_MSA, Not Principle City",household_children,h1n1_concern,"income_poverty_<= $75,000, Above Poverty",age_group_55 - 64 Years,hhs_geo_region_bhuqouqj,marital_status_Not Married,seasonal_vaccine
0,0.0,2.0,0,2.0,0,0.0,0.0,0,1,0.0,...,0.0,1.0,0,0.0,1.0,0,1,0,1,0
1,2.0,4.0,1,4.0,0,0.0,0.0,0,1,0.0,...,0.0,1.0,1,0.0,3.0,0,0,1,1,1
3,1.0,5.0,1,5.0,0,1.0,0.0,1,1,0.0,...,1.0,0.0,0,0.0,1.0,0,0,0,1,1
4,1.0,3.0,0,2.0,0,1.0,0.0,0,1,0.0,...,0.0,1.0,1,0.0,2.0,1,0,0,0,0
5,1.0,5.0,0,1.0,0,0.0,0.0,1,1,0.0,...,1.0,1.0,0,3.0,3.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26701,2.0,4.0,1,4.0,0,0.0,0.0,0,1,1.0,...,0.0,1.0,0,0.0,2.0,0,0,0,1,0
26702,0.0,5.0,0,1.0,0,0.0,0.0,1,1,0.0,...,0.0,0.0,0,0.0,2.0,1,0,0,1,0
26703,2.0,5.0,1,2.0,1,0.0,1.0,0,1,0.0,...,1.0,0.0,0,0.0,1.0,1,0,0,1,0
26704,2.0,5.0,0,2.0,1,1.0,0.0,0,1,0.0,...,0.0,1.0,1,0.0,2.0,0,1,0,1,1


In [136]:
cat_preprocessing = Pipeline([
    ("ohe", OneHotEncoder(categories="auto", handle_unknown="ignore"))
])

numeric_preprocessing =   Pipeline([
    ('standard_scaler', StandardScaler())
])

numeric_prep_top 

In [178]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_preprocessing, top_feat_numeric),
    ], remainder='passthrough'
)
preprocessor_top = ColumnTransformer(
    transformers=[
        ("num", numeric_preprocessing, top_feat_numeric),
    ], remainder='passthrough'
)

In [113]:
ohe = OneHotEncoder()
ohe.fit(X_train[df_object.columns])
ohe.get_feature_names(df_object.columns)
feature_names = list(df_numeric.columns)+ list(ohe.get_feature_names(df_object.columns))
feature_names

['h1n1_concern',
 'h1n1_knowledge',
 'behavioral_antiviral_meds',
 'behavioral_avoidance',
 'behavioral_face_mask',
 'behavioral_wash_hands',
 'behavioral_large_gatherings',
 'behavioral_outside_home',
 'behavioral_touch_face',
 'doctor_recc_h1n1',
 'doctor_recc_seasonal',
 'chronic_med_condition',
 'child_under_6_months',
 'health_worker',
 'opinion_h1n1_vacc_effective',
 'opinion_h1n1_risk',
 'opinion_h1n1_sick_from_vacc',
 'opinion_seas_vacc_effective',
 'opinion_seas_risk',
 'opinion_seas_sick_from_vacc',
 'household_adults',
 'household_children',
 'respondent_id',
 'h1n1_vaccine',
 'seasonal_vaccine',
 'age_group_18 - 34 Years',
 'age_group_35 - 44 Years',
 'age_group_45 - 54 Years',
 'age_group_55 - 64 Years',
 'age_group_65+ Years',
 'education_12 Years',
 'education_< 12 Years',
 'education_College Graduate',
 'education_Some College',
 'race_Black',
 'race_Hispanic',
 'race_Other or Multiple',
 'race_White',
 'sex_Female',
 'sex_Male',
 'income_poverty_<= $75,000, Above Pover

In [138]:
X=df_final.drop(columns=['respondent_id', 'h1n1_vaccine'])
y=df_final['h1n1_vaccine']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [85]:
X_h = df_health.drop(columns=['respondent_id', 'h1n1_vaccine'])
y_h = df_health['h1n1_vaccine']
X_h_train, X_h_test, y_h_train, y_h_test = train_test_split(X_h, y_h, random_state=42)

In [139]:
X_trans = preprocessor.fit_transform(X_train)

In [144]:
pipe = Pipeline([
    ("preprocessing", preprocessor),
    ('logreg', LogisticRegression(C=5, solver='sag', max_iter=1000, random_state=69))
])

In [145]:
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.8537604456824512

In [146]:
cross_val_score(pipe, X_test, y_test)

array([0.845953  , 0.85813751, 0.84943429, 0.84073107, 0.85801394])

In [106]:
list(X_train.columns)

['h1n1_concern',
 'h1n1_knowledge',
 'behavioral_antiviral_meds',
 'behavioral_avoidance',
 'behavioral_face_mask',
 'behavioral_wash_hands',
 'behavioral_large_gatherings',
 'behavioral_outside_home',
 'behavioral_touch_face',
 'doctor_recc_h1n1',
 'doctor_recc_seasonal',
 'chronic_med_condition',
 'child_under_6_months',
 'health_worker',
 'opinion_h1n1_vacc_effective',
 'opinion_h1n1_risk',
 'opinion_h1n1_sick_from_vacc',
 'opinion_seas_vacc_effective',
 'opinion_seas_risk',
 'opinion_seas_sick_from_vacc',
 'age_group',
 'education',
 'race',
 'sex',
 'income_poverty',
 'marital_status',
 'rent_or_own',
 'employment_status',
 'hhs_geo_region',
 'census_msa',
 'household_adults',
 'household_children',
 'seasonal_vaccine']

In [179]:
X_top = df_top_feat.drop('h1n1_vaccine', axis=1)
y_top = df_top_feat['h1n1_vaccine']

X_train_top, X_test_top, y_train_top, y_test_top = train_test_split(X_top, y_top, random_state=42)

In [180]:
pipe2 = Pipeline([
    ("preprocessing", preprocessor_top),
    ('logreg', LogisticRegression(C=5, solver='sag', max_iter=1000, random_state=69))
])

In [181]:
pipe2.fit(X_train_top, y_train_top)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('standard_scaler',
                                                                   StandardScaler())]),
                                                  ['h1n1_concern',
                                                   'h1n1_knowledge',
                                                   'behavioral_large_gatherings',
                                                   'behavioral_touch_face',
                                                   'doctor_recc_h1n1',
                                                   'doctor_recc_seasonal',
                                                   'chronic_med_condition',
                                                   'child_under_6_months',
                                                   'health_worker',
                

In [183]:
pipe2.score(X_test_top, y_test_top)

0.8450146375064578

In [190]:
cross_val_score(pipe2, X_test_top, y_test_top, scoring='precision')

array([0.725     , 0.70833333, 0.69832402, 0.67204301, 0.66502463])

In [189]:
cross_val_score(pipe, X_test, y_test, scoring='precision')

array([0.70243902, 0.74871795, 0.70046083, 0.68599034, 0.73429952])

15316    1
1462     0
10655    1
14734    1
19905    0
        ..
13896    0
25081    0
6277     1
990      1
18373    1
Name: h1n1_vaccine, Length: 17232, dtype: int64

In [201]:
lr_sm = sm.Logit(endog=y_train, exog=X_train).fit()


Optimization terminated successfully.
         Current function value: 0.355061
         Iterations 7


LinAlgError: Singular matrix

In [198]:
lr_sm_top = sm.Logit(y_train_top, X_train_top).fit()
lr_sm_top.summary()

Optimization terminated successfully.
         Current function value: 0.354103
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           h1n1_vaccine   No. Observations:                17420
Model:                          Logit   Df Residuals:                    17385
Method:                           MLE   Df Model:                           34
Date:                Wed, 09 Mar 2022   Pseudo R-squ.:                  0.3405
Time:                        12:31:00   Log-Likelihood:                -6168.5
converged:                       True   LL-Null:                       -9353.4
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================================
                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
h1n1_knowledge                               0.0430      0.042      1.022      0.307      -0.039       0.125
opinion_seas_vacc_effective                 -0.1012      0.029     -3.455      0.001      -0.159      -0.044
rent_or_own_Rent                            -0.0481      0.063     -0.768      0.442      -0.171       0.075
opinion_h1n1_sick_from_vacc                  0.0095      0.020      0.471      0.638      -0.030       0.049
hhs_geo_region_lzgpxyit                     -0.1613      0.066     -2.454      0.014      -0.290      -0.032
behavioral_large_gatherings                 -0.2150      0.051     -4.183      0.000      -0.316      -0.114
health_worker                                0.6759      0.067     10.067      0.000       0.544       0.807
age_group_65+ Years                          0.1321      0.075      1.770      0.077      -0.014       0.278
race_White                                  -0.0599      0.064     -0.943      0.346      -0.184       0.065
child_under_6_months                         0.2206      0.079      2.792      0.005       0.066       0.375
doctor_recc_h1n1                             2.2190      0.066     33.555      0.000       2.089       2.349
sex_Male                                    -5.8248      0.242    -24.066      0.000      -6.299      -5.350
opinion_h1n1_risk                            0.4074      0.022     18.729      0.000       0.365       0.450
income_poverty_> $75,000                    -0.1466      0.078     -1.886      0.059      -0.299       0.006
education_< 12 Years                        -0.2499      0.093     -2.676      0.007      -0.433      -0.067
sex_Female                                  -5.9977      0.245    -24.497      0.000      -6.478      -5.518
marital_status_Married                       0.1618      0.190      0.851      0.395      -0.211       0.534
opinion_seas_risk                            0.0064      0.021      0.300      0.764      -0.035       0.048
employment_status_Employed                  -0.0241      0.105     -0.229      0.819      -0.231       0.182
opinion_seas_sick_from_vacc                 -0.0233      0.020     -1.172      0.241      -0.062       0.016
household_adults                             0.0109      0.035      0.308      0.758      -0.059       0.080
chronic_med_condition                        0.0129      0.052      0.251      0.802      -0.088       0.114
employment_status_Not in Labor Force        -0.0205      0.108     -0.189      0.850      -0.233       0.192
opinion_h1n1_vacc_effective                  0.6920      0.032     21.508      0.000       0.629       0.755
education_College Graduate                   0.0347      0.053      0.661      0.509      -0.068       0.138
doctor_recc_seasonal                        -1.0065      0.065    -15.449      0.000      -1.134      -0.879
behavioral_touch_face                       -0.0300      0.054     -0.552      0

In [205]:
insig_feat = ['marital_status_Not Married','employment_status_Not in Labor Force', 'employment_status_Employed'
'chronic_med_condition','opinion_seas_risk','household_adults','hhs_geo_region_bhuqouqj'
, 'opinion_h1n1_sick_from_vacc','behavioral_touch_face', 'education_College Graduate', 
'household_children', 'rent_or_own_Rent', 'marital_status_Married', 'race_White', 'h1n1_knowledge',
'opinion_seas_sick_from_vacc']


In [203]:
rfc2 = RandomForestClassifier()
rfc2.fit(X_train, y_train)
rfc2.score(X_test,y_test)

0.8448816155988857

In [204]:
rfc_top = RandomForestClassifier()
rfc_top.fit(X_train_top, y_train_top)
rfc_top.score(X_test_top,y_test_top)

0.8436369898398485

In [206]:
top_feat_sig = top_feat_list
top_feat_sig = top_feat_sig.remove(insig_feat)

ValueError: list.remove(x): x not in list

In [215]:
top_feat_sig =[x for x in top_feat_list if x not in insig_feat]
top_feat_sig

['opinion_seas_vacc_effective',
 'hhs_geo_region_lzgpxyit',
 'behavioral_large_gatherings',
 'health_worker',
 'age_group_65+ Years',
 'child_under_6_months',
 'doctor_recc_h1n1',
 'sex_Male',
 'opinion_h1n1_risk',
 'income_poverty_> $75,000',
 'education_< 12 Years',
 'sex_Female',
 'employment_status_Employed',
 'chronic_med_condition',
 'opinion_h1n1_vacc_effective',
 'doctor_recc_seasonal',
 'census_msa_MSA, Not Principle  City',
 'h1n1_concern',
 'income_poverty_<= $75,000, Above Poverty',
 'age_group_55 - 64 Years',
 'seasonal_vaccine',
 'h1n1_vaccine']

In [218]:
X_ts = X_top[top_feat_sig]
y_ts = y_top

X_train_ts, X_test_ts, y_train_ts, y_test_ts = train_test_split(X_ts, y_ts, test_size=0.2, random_state=42)

In [220]:
lr_sig = LogisticRegression(C=5, max_iter=1000, solver='sag')
lr_sig.fit(X_train_ts, y_train_ts)
lr_sig.score(X_test_ts, y_test_ts)

0.8450279810589755

In [224]:
lr_sm_sig = sm.Logit(y_train_ts, X_train_ts).fit()
lr_sm_sig.summary()


Optimization terminated successfully.
         Current function value: 0.355961
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           h1n1_vaccine   No. Observations:                18581
Model:                          Logit   Df Residuals:                    18560
Method:                           MLE   Df Model:                           20
Date:                Wed, 09 Mar 2022   Pseudo R-squ.:                  0.3366
Time:                        12:57:13   Log-Likelihood:                -6614.1
converged:                       True   LL-Null:                       -9969.7
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================================
                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
opinion_seas_vacc_effective                 -0.1103      0.028     -3.964      0.000      -0.165      -0.056
hhs_geo_region_lzgpxyit                     -0.1417      0.063     -2.260      0.024      -0.265      -0.019
behavioral_large_gatherings                 -0.2285      0.048     -4.720      0.000      -0.323      -0.134
health_worker                                0.6874      0.064     10.718      0.000       0.562       0.813
age_group_65+ Years                          0.1145      0.063      1.832      0.067      -0.008       0.237
child_under_6_months                         0.2412      0.076      3.164      0.002       0.092       0.391
doctor_recc_h1n1                             2.2020      0.064     34.508      0.000       2.077       2.327
sex_Male                                    -5.7705      0.155    -37.199      0.000      -6.074      -5.466
opinion_h1n1_risk                            0.4051      0.018     21.930      0.000       0.369       0.441
income_poverty_> $75,000                    -0.0396      0.068     -0.580      0.562      -0.173       0.094
education_< 12 Years                        -0.2797      0.087     -3.207      0.001      -0.451      -0.109
sex_Female                                  -5.9587      0.155    -38.336      0.000      -6.263      -5.654
employment_status_Employed                  -0.0045      0.054     -0.083      0.934      -0.110       0.101
chronic_med_condition                        0.0288      0.050      0.582      0.561      -0.068       0.126
opinion_h1n1_vacc_effective                  0.6783      0.031     22.053      0.000       0.618       0.739
doctor_recc_seasonal                        -0.9997      0.063    -15.909      0.000      -1.123      -0.876
census_msa_MSA, Not Principle  City         -0.0982      0.045     -2.199      0.028      -0.186      -0.011
h1n1_concern                                -0.1191      0.028     -4.196      0.000      -0.175      -0.063
income_poverty_<= $75,000, Above Poverty    -0.0916      0.059     -1.558      0.119      -0.207       0.024
age_group_55 - 64 Years                      0.2229      0.058      3.876      0.000       0.110       0.336
seasonal_vaccine                             1.9717      0.055     35.835      0.000       1.864       2.080
============================================================================================================
"""

In [228]:
top_feat_sig_2 = top_feat_sig
top_feat_sig_2.remove('income_poverty_> $75,000')
top_feat_sig_2.remove('employment_status_Employed')
top_feat_sig_2.remove('chronic_med_condition')


ValueError: list.remove(x): x not in list

In [261]:
remove_feat = ['income_poverty_> $75,000','employment_status_Employed'
,'chronic_med_condition', 'sex_Male']
top_feat_sig_final = [x for x in top_feat_sig if x not in remove_feat]
top_feat_sig_final

['opinion_seas_vacc_effective',
 'hhs_geo_region_lzgpxyit',
 'behavioral_large_gatherings',
 'health_worker',
 'age_group_65+ Years',
 'child_under_6_months',
 'doctor_recc_h1n1',
 'opinion_h1n1_risk',
 'education_< 12 Years',
 'sex_Female',
 'opinion_h1n1_vacc_effective',
 'doctor_recc_seasonal',
 'census_msa_MSA, Not Principle  City',
 'h1n1_concern',
 'income_poverty_<= $75,000, Above Poverty',
 'age_group_55 - 64 Years',
 'seasonal_vaccine']

In [262]:
X_ts2 = X_top[top_feat_sig_final]
y_ts2 = y_top

X_train_ts2, X_test_ts2, y_train_ts2, y_test_ts2 = train_test_split(X_ts2, y_ts2, test_size=0.2, random_state=42)

In [263]:
lr_sig_2 = LogisticRegression(C=5, max_iter=1000, solver='sag')
lr_sig_2.fit(X_train_ts2, y_train_ts2)
lr_sig_2.score(X_test_ts2, y_test_ts2)

0.8452432199741713

In [264]:
cross_val_score(lr_sig_2, X_train_ts2, y_train_ts2), cross_val_score(lr_sig_2, X_test_ts2, y_test_ts2)

(array([0.84853376, 0.84607104, 0.84472551, 0.85145318, 0.85091496]),
 array([0.85806452, 0.8320775 , 0.83853606, 0.83853606, 0.84607104]))

In [257]:
cross_val_score(lr_sig_2, X_train_ts2, y_train_ts2).mean(), cross_val_score(lr_sig_2, X_test_ts2, y_test_ts2).mean()

(0.8483396914013032, 0.8426570367026633)

In [252]:
rfc_sig_2 = RandomForestClassifier()
rfc_sig_2.fit(X_train_ts2, y_train_ts2)
rfc_sig_2.score(X_test_ts2, y_test_ts2)

0.8297460180800689

In [253]:
cross_val_score(rfc_sig_2, X_train_ts2, y_train_ts2), cross_val_score(rfc_sig_2, X_test_ts2, y_test_ts2)

(array([0.82324455, 0.82077503, 0.82723358, 0.82400431, 0.83261572]),
 array([0.82258065, 0.80839612, 0.83530678, 0.82884822, 0.82884822]))

In [254]:
w_avg = VotingClassifier(estimators=[
    ('lr', lr_sig_2),
    ('rfc', rfc_sig_2)],
    weights=[0.9, 0.1])
w_avg.fit(X_train_ts2, y_train_ts2)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=5, max_iter=1000,
                                                 solver='sag')),
                             ('rfc', RandomForestClassifier())],
                 weights=[0.9, 0.1])

In [255]:
w_avg.score(X_test_ts2, y_test_ts2)

0.8452432199741713

In [246]:
cross_val_score(w_avg, X_train_ts2, y_train_ts2), cross_val_score(w_avg, X_test_ts2, y_test_ts2)

(array([0.84772666, 0.84607104, 0.84472551, 0.85145318, 0.85037675]),
 array([0.86021505, 0.83638321, 0.84499462, 0.84284177, 0.84930032]))

In [256]:
cross_val_score(w_avg, X_train_ts2, y_train_ts2).mean(), cross_val_score(w_avg, X_test_ts2, y_test_ts2).mean()

(0.8483396914013032, 0.8426570367026633)

In [247]:
y_train.value_counts(normalize=True)

0    0.774547
1    0.225453
Name: h1n1_vaccine, dtype: float64

In [265]:
lr_sm_final = sm.Logit(y_train_ts2, X_train_ts2).fit()
lr_sm_final.summary()

Optimization terminated successfully.
         Current function value: 0.416386
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           h1n1_vaccine   No. Observations:                18581
Model:                          Logit   Df Residuals:                    18564
Method:                           MLE   Df Model:                           16
Date:                Wed, 09 Mar 2022   Pseudo R-squ.:                  0.2240
Time:                        13:29:27   Log-Likelihood:                -7736.9
converged:                       True   LL-Null:                       -9969.7
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================================
                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
opinion_seas_vacc_effective                 -0.5565      0.021    -25.889      0.000      -0.599      -0.514
hhs_geo_region_lzgpxyit                     -0.3290      0.058     -5.640      0.000      -0.443      -0.215
behavioral_large_gatherings                 -0.2464      0.045     -5.537      0.000      -0.334      -0.159
health_worker                                0.5492      0.058      9.411      0.000       0.435       0.664
age_group_65+ Years                         -0.1180      0.052     -2.267      0.023      -0.220      -0.016
child_under_6_months                         0.0307      0.071      0.434      0.665      -0.108       0.170
doctor_recc_h1n1                             2.2792      0.062     36.766      0.000       2.158       2.401
opinion_h1n1_risk                            0.3269      0.018     18.669      0.000       0.293       0.361
education_< 12 Years                        -0.4978      0.080     -6.219      0.000      -0.655      -0.341
sex_Female                                  -0.5076      0.041    -12.285      0.000      -0.589      -0.427
opinion_h1n1_vacc_effective                 -0.0214      0.021     -1.024      0.306      -0.062       0.020
doctor_recc_seasonal                        -1.0257      0.060    -16.999      0.000      -1.144      -0.907
census_msa_MSA, Not Principle  City         -0.4160      0.041    -10.268      0.000      -0.495      -0.337
h1n1_concern                                -0.1959      0.026     -7.649      0.000      -0.246      -0.146
income_poverty_<= $75,000, Above Poverty    -0.4057      0.040    -10.067      0.000      -0.485      -0.327
age_group_55 - 64 Years                     -0.0320      0.052     -0.612      0.540      -0.134       0.070
seasonal_vaccine                             2.1715      0.052     41.824      0.000       2.070       2.273
============================================================================================================
"""

In [268]:
df['child_under_6_months'].value_counts(normalize=True)

0.0    0.916149
1.0    0.083851
Name: child_under_6_months, dtype: float64

In [269]:
len(top_feat_sig_final)

17